In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate

from adm1.ode import ADM1_ODE
#from adm1.dae import DAESolve  # pure DAE solver
from adm1.params import *
import scipy

499.995 0.0049999999999954525 50.0
10000
880.0
State Zero Values:
S_su: 1e-05
S_aa: 1e-05
S_fa: 1e-05
S_va: 1e-05
S_bu: 1e-05
S_pro: 1e-05
S_ac: 1e-05
S_h2: 1e-08
S_ch4: 1e-05
S_IC: 0.04
S_IN: 0.01
S_I: 0.02
X_xc1: 6.2499375e-07
X_ch1: 1.5624843750000003e-06
X_pr1: 6.249937500000001e-06
X_li1: 1.5624843750000003e-06
X_xc2: 0.0
X_ch2: 7.894179486649867e-11
X_pr2: 1.8391512794263297e-11
X_li2: 2.6666923387829425e-12
X_su: 1.41
X_aa: 0.75
X_fa: 0.38
X_c4: 0.35
X_pro: 9.85
X_ac: 47.5
X_h2: 0.466
X_I: 25.6
S_cation: 0.04
S_anion: 0.02
S_H_ion: 6.04e-08
S_va_ion: 0.0082
S_bu_ion: 0.0156
S_pro_ion: 0.0172
S_ac_ion: 0.11199
S_hco3_ion: 0.15415
S_co2: -0.11415
S_nh3: 0.0025
S_nh4_ion: 0.0075
S_gas_h2: 4.91e-06
S_gas_ch4: 1.78
S_gas_co2: 0.025

State Input Values:
S_su_in: 0.001
S_aa_in: 0.001
S_fa_in: 0.001
S_va_in: 0.001
S_bu_in: 0.001
S_pro_in: 0.001
S_ac_in: 0.001
S_h2_in: 1e-08
S_ch4_in: 1e-05
S_IC_in: 0.04
S_IN_in: 0.01
S_I_in: 0.02
X_xc1_in: 0.0
X_ch1_in: 20.056799429999998
X_pr1_in: 12.6

In [2]:
# Initiate the cache data frame for storing simulation results
simulate_results = pd.DataFrame([state_zero])
columns = ["S_su", "S_aa", "S_fa", "S_va", "S_bu", "S_pro", "S_ac", "S_h2", "S_ch4", "S_IC", "S_IN", "S_I", "X_xc1", "X_ch1", "X_pr1", "X_li1", "X_xc2", "X_ch2", "X_pr2", "X_li2", "X_su", "X_aa", "X_fa", "X_c4", "X_pro", "X_ac", "X_h2", "X_I", "S_cation", "S_anion", "pH", "S_va_ion", "S_bu_ion", "S_pro_ion", "S_ac_ion", "S_hco3_ion", "S_co2", "S_nh3", "S_nh4_ion", "S_gas_h2", "S_gas_ch4", "S_gas_co2"]
simulate_results.columns = columns

In [3]:

# Initiate the cache data frame for storing simulation results
simulate_results = pd.DataFrame([state_zero])
columns = ["S_su", "S_aa", "S_fa", "S_va", "S_bu", "S_pro", "S_ac", "S_h2", "S_ch4", "S_IC", "S_IN", "S_I", "X_xc1", "X_ch1", "X_pr1", "X_li1", "X_xc2", "X_ch2", "X_pr2", "X_li2", "X_su", "X_aa", "X_fa", "X_c4", "X_pro", "X_ac", "X_h2", "X_I", "S_cation", "S_anion", "pH", "S_va_ion", "S_bu_ion", "S_pro_ion", "S_ac_ion", "S_hco3_ion", "S_co2", "S_nh3", "S_nh4_ion", "S_gas_h2", "S_gas_ch4", "S_gas_co2"]
simulate_results.columns = columns

In [4]:
# Initiate cache data frame for storing gasflow values
initflow = {'p_gas_h2': [0],'p_gas_ch4': [0],'p_gas_co2': [0],'p_gas': [0],'q_gas': [0], 'q_ch4': [0], 'total_ch4': [0],'p_gas_ch4/p_gas': [0],'p_gas_co2/p_gas': [0], 'ch4_yield':[0], 'co2_yield':[0] }
gasflow = pd.DataFrame(initflow)
total_ch4 = 0

In [5]:
# Initiate cache data frame for storing inhibition values
initflow = {'I_5': [0],'I_7': [0],'I_8': [0],'I_10': [0],'I_12': [0],'I_pH_aa': [0],'I_pH_h2': [0],'I_IN_lim': [0],'I_h2_fa': [0],'I_h2_c4': [0], 'I_h2_pro': [0]}
inhibition = pd.DataFrame(initflow)

In [6]:
# Initiate cache data frame for storing ions values
initflow = {'S_cation': [0],'S_anion': [0],'S_H_ion': [0],'S_va_ion': [0],'S_bu_ion': [0],'S_pro_ion': [0],'S_ac_ion': [0],'S_hco3_ion': [0],'S_nh4_ion': [0]}
ions = pd.DataFrame(initflow)

In [7]:
def DAESolve():
  global S_va_ion, S_bu_ion, S_pro_ion, S_ac_ion, S_hco3_ion, S_nh3, S_H_ion, pH, p_gas_h2, S_h2
  
  ##  DAE calculations 
  eps = 0.0000001
  
  prevS_H_ion = S_H_ion
  

  #initial values for Newton-Raphson solver parameter
  shdelta = 1.0
  shgradeq = 1.0
  S_h2delta = 1.0
  S_h2gradeq = 1.0
  tol = 10 ** (-12) #solver accuracy tolerance
  maxIter = 1000 #maximum number of iterations for solver
  i = 1
  j = 1
  
  ## DAE solver for S_H_ion from Rosen et al. (2006)
  while ((shdelta > tol or shdelta < -tol) and (i <= maxIter)):
    S_va_ion = K_a_va * S_va / (K_a_va + S_H_ion)
    S_bu_ion = K_a_bu * S_bu / (K_a_bu + S_H_ion)
    S_pro_ion = K_a_pro * S_pro / (K_a_pro + S_H_ion)
    S_ac_ion = K_a_ac * S_ac / (K_a_ac + S_H_ion)
    S_hco3_ion = K_a_co2 * S_IC / (K_a_co2 + S_H_ion)
    S_nh3 = K_a_IN * S_IN / (K_a_IN + S_H_ion)
    shdelta = S_cation + (S_IN - S_nh3) + S_H_ion - S_hco3_ion - S_ac_ion / 64.0 - S_pro_ion / 112.0 - S_bu_ion / 160.0 - S_va_ion / 208.0 - K_w / S_H_ion - S_anion
    shgradeq = 1 + K_a_IN * S_IN / ((K_a_IN + S_H_ion) * (K_a_IN + S_H_ion)) + K_a_co2 * S_IC / ((K_a_co2 + S_H_ion) * (K_a_co2 + S_H_ion)) \
              + 1 / 64.0 * K_a_ac * S_ac / ((K_a_ac + S_H_ion) * (K_a_ac + S_H_ion)) \
              + 1 / 112.0 * K_a_pro * S_pro / ((K_a_pro + S_H_ion) * (K_a_pro + S_H_ion)) \
              + 1 / 160.0 * K_a_bu * S_bu / ((K_a_bu + S_H_ion) * (K_a_bu + S_H_ion)) \
              + 1 / 208.0 * K_a_va * S_va / ((K_a_va + S_H_ion) * (K_a_va + S_H_ion)) \
              + K_w / (S_H_ion * S_H_ion)
    S_H_ion = S_H_ion - shdelta / shgradeq
    if S_H_ion <= 0:
        S_H_ion = tol
    i+=1
  
  # pH calculation
  pH = - np.log10(S_H_ion)
  
  #DAE solver for S_h2 from Rosen et al. (2006) 
  while ((S_h2delta > tol or S_h2delta < -tol) and (j <= maxIter)):
    
    #I_pH_aa = (K_pH_aa ** nn_aa) / (prevS_H_ion ** nn_aa + K_pH_aa ** nn_aa)
    #I_pH_h2 = (K_pH_h2 ** n_h2) / (prevS_H_ion ** n_h2 + K_pH_h2 ** n_h2)
    #I_IN_lim = 1 / (1 + (K_S_IN / S_IN))
    #I_h2_fa = 1 / (1 + (S_h2 / K_I_h2_fa))
    #I_h2_c4 = 1 / (1 + (S_h2 / K_I_h2_c4))
    #I_h2_pro = 1 / (1 + (S_h2 / K_I_h2_pro))
    
    
    I_pH_aa = 1
    I_pH_h2 = 1
    I_IN_lim = 1 
    I_h2_fa = 1
    I_h2_c4 = 1
    I_h2_pro = 1
  
    I_5 = I_pH_aa * I_IN_lim
    I_6 = I_5
    I_7 = I_pH_aa * I_IN_lim * I_h2_fa
    I_8 = I_pH_aa * I_IN_lim * I_h2_c4
    I_9 = I_8
    I_10 = I_pH_aa * I_IN_lim * I_h2_pro
  
    I_12 = I_pH_h2 * I_IN_lim
    

    
    Rho_5 = k_m_su * (S_su / (K_S_su + S_su)) * X_su * I_5  # Uptake of sugars
    Rho_6 = k_m_aa * (S_aa / (K_S_aa + S_aa)) * X_aa * I_6  # Uptake of amino-acids
    Rho_7 = k_m_fa * (S_fa / (K_S_fa + S_fa)) * X_fa * I_7  # Uptake of LCFA (long-chain fatty acids)
    Rho_8 = k_m_c4 * (S_va / (K_S_c4 + S_va)) * X_c4 * (S_va / (S_bu + S_va+ 1e-6)) * I_8  # Uptake of valerate
    Rho_9 = k_m_c4 * (S_bu / (K_S_c4 + S_bu)) * X_c4 * (S_bu / (S_bu + S_va+ 1e-6)) * I_9  # Uptake of butyrate
    Rho_10 = k_m_pro * (S_pro / (K_S_pro + S_pro)) * X_pro * I_10  # Uptake of propionate
    Rho_12 = k_m_h2 * (S_h2 / (K_S_h2 + S_h2)) * X_h2 * I_12  # Uptake of hydrogen
    p_gas_h2 = S_gas_h2 * R * T_ad / 16
    Rho_T_8 = k_L_a * (S_h2 - 16 * K_H_h2 * p_gas_h2)
    S_h2delta = q_ad / V_liq * (S_h2_in - S_h2) + (1 - Y_su) * f_h2_su * Rho_5 + (1 - Y_aa) * f_h2_aa * Rho_6 + (1 - Y_fa) * 0.3 * Rho_7 + (1 - Y_c4) * 0.15 * Rho_8 + (1 - Y_c4) * 0.2 * Rho_9 + (1 - Y_pro) * 0.43 * Rho_10 - Rho_12 - Rho_T_8
    S_h2gradeq = - 1.0 / V_liq * q_ad - 3.0 / 10.0 * (1 - Y_fa) * k_m_fa * S_fa / (K_S_fa + S_fa) * X_fa * I_pH_aa / (1 + K_S_IN / S_IN) / ((1 + S_h2 / K_I_h2_fa) * (1 + S_h2 / K_I_h2_fa)) / K_I_h2_fa - 3.0 / 20.0 * (1 - Y_c4) * k_m_c4 * S_va * S_va / (K_S_c4 + S_va) * X_c4 / (S_bu + S_va + eps) * I_pH_aa / (1 + K_S_IN / S_IN) / ((1 + S_h2 / K_I_h2_c4 ) * (1 + S_h2 / K_I_h2_c4 )) / K_I_h2_c4 - 1.0 / 5.0 * (1 - Y_c4) * k_m_c4 * S_bu * S_bu / (K_S_c4 + S_bu) * X_c4 / (S_bu + S_va + eps) * I_pH_aa / (1 + K_S_IN / S_IN) / ((1 + S_h2 / K_I_h2_c4 ) * (1 + S_h2 / K_I_h2_c4 )) / K_I_h2_c4 - 43.0 / 100.0 * (1 - Y_pro) * k_m_pro * S_pro / (K_S_pro + S_pro) * X_pro * I_pH_aa / (1 + K_S_IN / S_IN) / ((1 + S_h2 / K_I_h2_pro ) * (1 + S_h2 / K_I_h2_pro )) / K_I_h2_pro - k_m_h2 / (K_S_h2 + S_h2) * X_h2 * I_pH_h2 / (1 + K_S_IN / S_IN) + k_m_h2 * S_h2 / ((K_S_h2 + S_h2) * (K_S_h2 + S_h2)) * X_h2 * I_pH_h2 / (1 + K_S_IN / S_IN) - k_L_a
    S_h2 = S_h2 - S_h2delta / S_h2gradeq
    if S_h2 <= 0:
        S_h2 = tol
    j+=1

In [8]:
def simulate(t_step, y0, state_input, solvermethod):
    def ode_func(t, y):
        return ADM1_ODE(t, y, state_input)
    r = scipy.integrate.solve_ivp(ode_func, t_step, y0, method=solvermethod)
    return r.y

In [9]:

solvermethod = 'DOP853'
#  solvermethod = 'BDF'

# Current dynamic state vector (start from params.state_zero copy to avoid accidental mutation of imported list)
current_state = list(state_zero)

for u in t[1:]:
    n += 1

    # Build (possibly time-varying) influent vector (currently constant)
    state_input = [S_su_in,S_aa_in,S_fa_in,S_va_in,S_bu_in,S_pro_in,S_ac_in,S_h2_in,S_ch4_in,S_IC_in,S_IN_in,S_I_in,
                   X_xc1_in,X_ch1_in,X_pr1_in,X_li1_in,X_xc2_in,X_ch2_in,X_pr2_in,X_li2_in,
                   X_su_in,X_aa_in,X_fa_in,X_c4_in,X_pro_in,X_ac_in,X_h2_in,X_I_in,S_cation_in,S_anion_in]

    # Integrate over this step
    tstep = [t0, u]
    sol = simulate(tstep, current_state, state_input, solvermethod)

    # Extract last column (final state at time u)
    last = sol[:, -1]
    (S_su, S_aa, S_fa, S_va, S_bu, S_pro, S_ac, S_h2, S_ch4, S_IC, S_IN, S_I,
     X_xc1, X_ch1, X_pr1, X_li1, X_xc2, X_ch2, X_pr2, X_li2,
     X_su, X_aa, X_fa, X_c4, X_pro, X_ac, X_h2, X_I,
     S_cation, S_anion, S_H_ion, S_va_ion, S_bu_ion, S_pro_ion, S_ac_ion, S_hco3_ion, S_co2, S_nh3, S_nh4_ion, S_gas_h2, S_gas_ch4, S_gas_co2) = last

    # Apply DAE algebraic correction if enabled
    if DAE_switch == 1:
        S_H_ion, pH, S_h2, S_va_ion, S_bu_ion, S_pro_ion, S_ac_ion, S_hco3_ion, S_nh3 = DAESolve(
            S_H_ion, S_va, S_bu, S_pro, S_ac, S_IC, S_IN, S_cation, S_anion,
            S_su, S_aa, S_fa, S_h2, X_su, X_aa, X_fa, X_c4, X_pro, X_h2
        )
        S_co2 = S_IC - S_hco3_ion
        S_nh4_ion = S_IN - S_nh3
    else:
        pH = -np.log10(S_H_ion)

    print(f"Time step {n} (day {u:.4f}) S_su={S_su:.4e} S_aa={S_aa:.4e} S_fa={S_fa:.4e} S_h2={S_h2:.4e} pH={pH:.2f}")

    # Rebuild state vector for next integration segment
    current_state = [S_su, S_aa, S_fa, S_va, S_bu, S_pro, S_ac, S_h2, S_ch4, S_IC, S_IN, S_I,
                     X_xc1, X_ch1, X_pr1, X_li1, X_xc2, X_ch2, X_pr2, X_li2,
                     X_su, X_aa, X_fa, X_c4, X_pro, X_ac, X_h2, X_I,
                     S_cation, S_anion, S_H_ion, S_va_ion, S_bu_ion, S_pro_ion, S_ac_ion, S_hco3_ion, S_co2, S_nh3, S_nh4_ion, S_gas_h2, S_gas_ch4, S_gas_co2]

    # Record row (replace pH column with computed pH)
    row = current_state.copy()
    # Insert pH at index corresponding to columns list (replace existing S_H_ion position already present). columns has pH at index 30.
    # columns ordering: ... S_anion, pH, S_va_ion ... In current_state index 30 is S_H_ion; we will not store S_H_ion separately; store pH instead for dataframe.
    row_for_df = row.copy()
    row_for_df[30] = pH

    dfstate_zero = pd.DataFrame([row_for_df], columns=columns)
    simulate_results = pd.concat([simulate_results, dfstate_zero], ignore_index=True)

    t0 = u

TypeError: ADM1_ODE() takes 2 positional arguments but 3 were given

In [ ]:
phlogarray = -1 * np.log10(simulate_results['pH'])
simulate_results['pH'] = phlogarray

In [ ]:

print('DAE_switch =', DAE_switch)

print ('S_su =', S_su, "\n",
         'S_aa =', S_aa, "\n",
         'S_fa =', S_fa, "\n",
         'S_va =', S_va, "\n",
         'S_bu =', S_bu, "\n",
         'S_pro =', S_pro, "\n",
         'S_ac =', S_ac, "\n",
         'S_h2 =', S_h2, "\n",
         'S_ch4 =', S_ch4, "\n",
         'S_IC =', S_IC, "\n",
         'S_IN =', S_IN, "\n",
         'S_I =', S_I, "\n",
         'X_xc1 =', X_xc1, "\n",
         'X_ch1 =', X_ch1, "\n",
         'X_pr1 =', X_pr1, "\n",
         'X_li1 =', X_li1, "\n",
         'X_xc2 =', X_xc2, "\n",
         'X_ch2 =', X_ch2, "\n",
         'X_pr2 =', X_pr2, "\n",
         'X_li2 =', X_li2, "\n",
         'X_su =', X_su, "\n",
         'X_aa =', X_aa, "\n",
         'X_fa =', X_fa, "\n",
         'X_c4 =', X_c4, "\n",
         'X_pro =', X_pro, "\n",
         'X_ac =', X_ac, "\n",
         'X_h2 =', X_h2, "\n",
         'X_I =', X_I, "\n",
         'S_cation =', S_cation,"\n", 
         'S_anion =', S_anion, "\n",
         'S_H_ion =', S_H_ion, "\n",
         'S_va_ion =', S_va_ion, "\n",
         'S_bu_ion =', S_bu_ion, "\n",
         'S_pro_ion =', S_pro_ion, "\n",
         'S_ac_ion =', S_ac_ion, "\n",
         'S_hco3_ion =', S_hco3_ion, "\n",
         'S_co2 =', S_co2, "\n",
         'S_nh3 =', S_nh3, "\n",
         'S_nh4_ion =', S_nh4_ion,"\n", 
         'S_gas_h2 =', S_gas_h2, "\n",
         'S_gas_ch4 =', S_gas_ch4, "\n",
         'S_gas_co2 =', S_gas_co2, "\n")